In [ ]:
import numpy as np
np.int = int
np.float = float
np.complex = complex

# TODO: replace with pp = pulseqzero.facade
import pypulseq as pp
import torch
import MRzeroCore as mr0

import matplotlib.pyplot as plt

In [ ]:
DEFAULT_SYSTEM = pp.Opts(
    max_grad=28, grad_unit='mT/m',
    max_slew=150, slew_unit='T/m/s',
    rf_ringdown_time=20e-6, rf_dead_time=100e-6,
    adc_dead_time=20e-6,
    grad_raster_time=50e-6
)

In [ ]:
def flash_2D(fov=200e-3, slice_thickness=8e-3,
             n_read=64, n_phase=64,
             flip_angle=10, phase_cycling=84,
             system=DEFAULT_SYSTEM):
    """Linear, cartesian 2D FLASH with TR = 26 ms + 50 us * n_phase"""
    rf, gz, gzr = pp.make_sinc_pulse(
        flip_angle=flip_angle * np.pi / 180, duration=1e-3,
        slice_thickness=slice_thickness, apodization=0.5, time_bw_product=4,
        return_gz=True, system=system
    )

    adc_dur = n_phase * 50e-6
    gx = pp.make_trapezoid(channel='x', flat_area=n_read / fov, flat_time=adc_dur, system=system)
    adc = pp.make_adc(num_samples=n_read, duration=adc_dur, delay=gx.rise_time, system=system)
    gx_pre = pp.make_trapezoid(channel='x', area=-0.5 * gx.area, duration=5e-3, system=system)
    gx_spoil = pp.make_trapezoid(channel='x', area=1.5 * gx.area, duration=2e-3, system=system)

    seq = pp.Sequence(system)
    for i in range(n_phase):
        phase = 0.5 * phase_cycling * (2 + i + i**2)
        rf.phase_offset = (phase % 360) * np.pi / 180
        adc.phase_offset = rf.phase_offset

        phenc = (i - n_phase // 2) / fov

        seq.add_block(rf, gz)
        seq.add_block(gzr, pp.make_delay(5e-3))
        gp = pp.make_trapezoid(channel='y', area=phenc, duration=5e-3, system=system)
        seq.add_block(gx_pre, gp)
        seq.add_block(adc, gx)
        gp = pp.make_trapezoid(channel='y', area=-phenc, duration=5e-3, system=system)
        seq.add_block(gx_spoil, gp)
        seq.add_block(pp.make_delay(10e-3))

    seq.set_definition('FOV', [fov, fov, slice_thickness])
    seq.set_definition('Name', 'gre')
    return seq

In [ ]:
seq = flash_2D()

ok, err = seq.check_timing()
if not ok:
    print("".join(e for e in err))

seq.plot()
seq.write("export.seq")

In [ ]:
phantom = mr0.VoxelGridPhantom.brainweb("../quantified_brain.npz")
phantom.plot()
sim_data = phantom.build()

seq0 = mr0.Sequence.import_file("export.seq")
seq0.plot_kspace_trajectory()

In [ ]:
graph = mr0.compute_graph(seq0, sim_data, 200, 1e-3)
signal = mr0.execute_graph(graph, seq0, sim_data, print_progress=False)

kspace = signal.reshape(64, 64).T
reco = torch.fft.fftshift(torch.fft.fft2(torch.fft.fftshift(kspace)))

plt.figure(figsize=(9, 9))

plt.subplot(311)
plt.title('ADC signal')
plt.plot(signal.real, label='real')
plt.plot(signal.imag, label='imag')
plt.grid()

plt.subplot(334)
plt.title('k-space')
mr0.util.imshow(np.abs(kspace.numpy()))
plt.axis("off")
plt.subplot(337)
plt.title('log. k-space')
mr0.util.imshow(np.log(np.abs(kspace.numpy())))
plt.axis("off")

plt.subplot(335)
plt.title('FFT-magnitude')
mr0.util.imshow(reco.abs(), vmin=0)
plt.axis("off")
plt.subplot(338)
plt.title('FFT-phase')
mr0.util.imshow(reco.angle(), vmin=-np.pi, vmax=np.pi, cmap="twilight")
plt.axis("off")

plt.subplot(336)
plt.title('phantom PD')
mr0.util.imshow(phantom.PD)
plt.axis("off")
plt.subplot(339)
plt.title('phantom B0')
mr0.util.imshow(phantom.B0)
plt.axis("off")

plt.show()